In [1]:
from google.colab import files
uploaded = files.upload()


Saving test.txt to test (1).txt
Saving train.txt to train (1).txt
Saving val.txt to val (1).txt


In [2]:
import pandas as pd

df = pd.read_csv('train.txt', sep=';', names=['text', 'emotion'])
df.head()


,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [3]:
!pip install transformers datasets scikit-learn
!pip install -U transformers


In [4]:
import pandas as pd

# Load files (uploaded via Google Colab)
train_df = pd.read_csv('train.txt', sep=';', names=['text', 'label'])
val_df   = pd.read_csv('val.txt', sep=';', names=['text', 'label'])
test_df  = pd.read_csv('test.txt', sep=';', names=['text', 'label'])

# Combine train + val for training
full_train_df = pd.concat([train_df, val_df], ignore_index=True)

print(full_train_df['label'].value_counts())


label
joy         6066
sadness     5216
anger       2434
fear        2149
love        1482
surprise     653
Name: count, dtype: int64


In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
full_train_df['label_encoded'] = label_encoder.fit_transform(full_train_df['label'])
test_df['label_encoded'] = label_encoder.transform(test_df['label'])

num_labels = len(label_encoder.classes_)


In [15]:
train_ds = train_ds.rename_column("label_encoded", "labels")
test_ds = test_ds.rename_column("label_encoded", "labels")


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.metrics import classification_report


In [16]:


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:


model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=num_labels
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import classification_report

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    report_to="none",            # disable wandb
    save_strategy="no",
    logging_strategy="no",

)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": (preds == labels).mean()
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)


In [24]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=4500, training_loss=0.17788975694444445, metrics={'train_runtime': 3517.984, 'train_samples_per_second': 10.233, 'train_steps_per_second': 1.279, 'total_flos': 9472338173952000.0, 'train_loss': 0.17788975694444445, 'epoch': 2.0})

In [25]:
predictions = trainer.predict(test_ds)
preds = np.argmax(predictions.predictions, axis=1)
print(classification_report(test_df['label_encoded'], preds, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

       anger       0.91      0.95      0.93       275
        fear       0.89      0.93      0.91       224
         joy       0.95      0.95      0.95       695
        love       0.85      0.84      0.84       159
     sadness       0.97      0.97      0.97       581
    surprise       0.87      0.62      0.73        66

    accuracy                           0.94      2000
   macro avg       0.91      0.88      0.89      2000
weighted avg       0.94      0.94      0.93      2000



In [28]:
import torch

def predict_emotion(text):
    # Get the same device the model is on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize and move inputs to the model's device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(dim=1).item()

    return label_encoder.classes_[prediction]


In [38]:
predict_emotion("I do not like this")


'anger'